In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-05-16 23:44:15--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-05-16 23:44:16 (6.06 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL_AWS_personal_care").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Setup config file location
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive')

Mounted at /content/drive


In [5]:
from config import aws_rds_endpoint
from config import aws_rds_password
# Configuration for RDS instance
jdbc_url = f"jdbc:postgresql://{aws_rds_endpoint}:5432/big_data_db"
mode_1="append"
config_1 = {"user":"postgres",
          "password": aws_rds_password,
          "driver":"org.postgresql.Driver"}
mode_2="overwrite"
config_2 = {"user":"postgres",
          "password": aws_rds_password,
          "truncate": "true",
          "driver":"org.postgresql.Driver"}

In [6]:
from pyspark import SparkFiles
#load software reviews data from s3.amazonaws.com
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

percare_df = spark.read.option("header","true").csv(SparkFiles.get("amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"), inferSchema=True, dateFormat="yyyy-MM-dd",sep="\t")
percare_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

# **Drop duplicates and incomplete rows**

In [7]:
percare_df = percare_df.drop("marketplace","product_category","verified_purchase","review_headline","review_body")
print(percare_df.count())
percare_df = percare_df.dropna()
print(percare_df.count())
percare_df = percare_df.dropDuplicates()
print(percare_df.count())

85981
85978
85978


# **Examine the schema**

In [8]:
percare_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: string (nullable = true)



# **Transfor Dataframe type**

In [9]:
import datetime
percare_df = percare_df.withColumn("review_date",percare_df["review_date"].cast("date"))
percare_df.show(5)

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   37501537|R23QUEX8A6ZGT5|B00HES9CMS|     318730927|Viva Naturals #1 ...|          5|            0|          0|   N| 2015-08-27|
|   33405988| R4BE55RS5NT8H|B00HES9CMS|     318730927|Viva Naturals #1 ...|          4|            0|          0|   N| 2015-08-26|
|   29991695|R26VH1I5MJ1FA3|B00OSAUYTA|     792155282|Psoriasis & Eczem...|          5|            0|          0|   N| 2015-08-24|
|   11229098|R2K8RBIYHER7WJ|B001V9QMMO|     618360222|Fixodent Denture ...|          5|            0|          0|   N| 2015-08-04|
|   10837312|R1ASFHMO6PUUCT|B00H9L7VIW|     851045898|boostULTIMATE - 6...|        

In [10]:
percare_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: date (nullable = true)



# **Create Dataframe for review_id_table**

In [11]:
review_id_df = percare_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R23QUEX8A6ZGT5|   37501537|B00HES9CMS|     318730927| 2015-08-27|
| R4BE55RS5NT8H|   33405988|B00HES9CMS|     318730927| 2015-08-26|
|R26VH1I5MJ1FA3|   29991695|B00OSAUYTA|     792155282| 2015-08-24|
|R2K8RBIYHER7WJ|   11229098|B001V9QMMO|     618360222| 2015-08-04|
|R1ASFHMO6PUUCT|   10837312|B00H9L7VIW|     851045898| 2015-08-03|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [12]:
# Drop duplicated review existed in sql database
review_id_df_pre = spark.read.jdbc(url=jdbc_url,table="review_id_table", properties=config_1)
review_id_df = review_id_df.join(review_id_df_pre, ["review_id"], "leftanti")
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10SFLSDK0T4VU|   18108829|B0002L5A8Y|     276226768| 2007-10-24|
|R12AALHK486N33|   43105079|B000EE5DTK|     315552854| 2009-08-18|
|R12ECLCPII68JO|   19930639|B001BLCZZU|     293902826| 2008-10-19|
|R136LVSFB94COK|   11750634|B004O23YE4|     284816422| 2015-04-07|
|R136Y5BHL9LH5D|   35525657|B0000YVAW8|     879193365| 2005-06-10|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [13]:
print(review_id_df.count())
review_id_df = review_id_df.dropDuplicates(["review_id"])
print(review_id_df.count())

85978
85978


# **Create Dataframe for products_table**

In [14]:
products_df = percare_df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00HES9CMS|Viva Naturals #1 ...|
|B00HES9CMS|Viva Naturals #1 ...|
|B00OSAUYTA|Psoriasis & Eczem...|
|B001V9QMMO|Fixodent Denture ...|
|B00H9L7VIW|boostULTIMATE - 6...|
+----------+--------------------+
only showing top 5 rows



In [15]:
# Drop duplicated review existed in sql database
products_df_pre = spark.read.jdbc(url=jdbc_url,table="products", properties=config_1)
products_df = products_df.join(products_df_pre, ["product_id"], "leftanti")
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0000532OW|Ezy Dose Ezy Crus...|
|B0000532OW|Ezy Dose Ezy Crus...|
|B0000532OW|Ezy Dose Ezy Crus...|
|B0000532OW|Ezy Dose Ezy Crus...|
|B0000532OW|Ezy Dose Ezy Crus...|
+----------+--------------------+
only showing top 5 rows



In [16]:
print(products_df.count())
products_df = products_df.dropDuplicates(["product_id"])
print(products_df.count())

85978
17562


# **Create Dataframe for customers_table**

In [17]:
customers_df_new = percare_df.select(["customer_id"])
customers_df_new = customers_df_new.groupBy("customer_id").count()
customers_df_new.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   11229098|    1|
|    2416824|    1|
|   11639065|    1|
|   29741671|    1|
|   16356076|    1|
+-----------+-----+
only showing top 5 rows



In [18]:
# Creat a copy of sql database
customers_df_pre = spark.read.jdbc(url=jdbc_url,table="customers", properties=config_1)
schema = customers_df_pre.schema
customers_df_pre_pd = customers_df_pre.toPandas()
_customers_df_pre = spark.createDataFrame(customers_df_pre_pd,schema=schema)
del customers_df_pre_pd

In [19]:
# Get the total counts of each customer
if _customers_df_pre.count():
  customers_df=customers_df_new.join(_customers_df_pre,on=["customer_id"],how="outer")
  customers_df=customers_df.fillna(0)
  customers_df=customers_df.withColumn('total', sum(customers_df[col] for col in ["count", "customer_count"]))
  customers_df = customers_df.drop("count", "customer_count")
  customers_df = customers_df.withColumnRenamed("total", "customer_count")
  customers_df.show(5)
else:
  customers_df = customers_df_new.withColumnRenamed("count", "customer_count")
  customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      28759|             1|
|      33375|             1|
|      45341|             1|
|     176073|             1|
|     235096|             1|
+-----------+--------------+
only showing top 5 rows



In [20]:
print(customers_df.count())

182774


# **Create Dataframe for vine_table**

In [21]:
vine_df = percare_df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R23QUEX8A6ZGT5|          5|            0|          0|   N|
| R4BE55RS5NT8H|          4|            0|          0|   N|
|R26VH1I5MJ1FA3|          5|            0|          0|   N|
|R2K8RBIYHER7WJ|          5|            0|          0|   N|
|R1ASFHMO6PUUCT|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [22]:
# Drop duplicated review existed in sql database
vine_df_pre = spark.read.jdbc(url=jdbc_url,table="vine_table", properties=config_1)
vine_df = vine_df.join(vine_df_pre, ["review_id"], "leftanti")
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10SFLSDK0T4VU|          5|            5|          5|   N|
|R12AALHK486N33|          5|            0|          0|   N|
|R12ECLCPII68JO|          5|            0|         11|   N|
|R136LVSFB94COK|          4|            0|          0|   N|
|R136Y5BHL9LH5D|          1|           76|         81|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [23]:
print(vine_df.count())
vine_df = vine_df.dropDuplicates(["review_id"])
print(vine_df.count())

85978
85978


# **Write DataFrame to RDS**

In [24]:
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode_1, properties=config_1)

In [25]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode_1, properties=config_1)

In [26]:
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode_2, properties=config_2)

In [27]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode_1, properties=config_1)